In [4]:
import urllib.request 
import urllib.parse
import json
import time
import concurrent.futures
import csv
import pandas as pd
import numpy as np

#importing all the necessary libraries

In [5]:
#all search suggestions as a Json file
y=urllib.request.urlopen("https://www.refinitiv.com/bin/esg/esgsearchsuggestions").read()
y=json.loads(y.decode('utf-8'))

#number of our search suggestions
length=len(y) 

#a funtion that returns the information of every company in a dictionary
def func(i):
    i=int(i)
    ricCode=y[i]["ricCode"]
    CompanyName=y[i]["companyName"]
    
    #getting the data of every company using it's ricCode(found the following link by inspecting the page)
    x=urllib.request.urlopen(f"https://www.refinitiv.com/bin/esg/esgsearchresult?ricCode={ricCode}").read()
    x=json.loads(x.decode('utf-8'))
    
    #checks if the dictionary containing our information is empty or not, in case it's empty, it creates an empty dictionary
    if len(x["esgScore"]['TR.TRESG'])!=0:
        Name=CompanyName
        ESG=x["esgScore"]['TR.TRESG']["score"]
        rank=x["industryComparison"]["rank"]
        totalIndustries=x["industryComparison"]["totalIndustries"]
        EnvironmentPillar=x["esgScore"]['TR.EnvironmentPillar']["score"] 
        SocialPillar=x["esgScore"]['TR.SocialPillar']["score"]
        GovernancePillar=x["esgScore"]['TR.GovernancePillar']["score"]

        info={
                "Name":Name,
                "ESG":ESG,
                "rank":rank,
                "totalIndustries":totalIndustries,
                "EnvironmentPillar": EnvironmentPillar,
                "SocialPillar":SocialPillar,
                "GovernancePillar":GovernancePillar,
                "ricCode":ricCode

                }
  
        
    else:
        info={
                "Name":"",
                "ESG":"",
                "rank":"",
                "totalIndustries":"",
                "EnvironmentPillar": "",
                "SocialPillar":"",
                "GovernancePillar":""

                }
    return(info)
        

    



NumList=range(0,length)
ThreadList=[]

start=time.perf_counter()

for i in NumList:
     with concurrent.futures.ThreadPoolExecutor() as executor:
         
        f1=executor.submit(func,i)
        ThreadList.append(f1.result())
finish=time.perf_counter()
print(finish-start)

#web crawler and the time it takes

8933.950659500202


In [6]:
#converting our list of dictionaries to a dataframe
df=pd.DataFrame(ThreadList)
df

,Name,ESG,rank,totalIndustries,EnvironmentPillar,SocialPillar,GovernancePillar,ricCode
0,Kimball Electronics Inc,53,84,232,37,83,22,KE.O
1,Importadora y Exportadora de la Patagonia SA,15,138,151,7,20,14,PATA.BA
2,Sociedad Comercial del Plata SA,12,364,410,0,12,30,COME.BA
3,Unipar Indupa SAIC,13,304,325,1,2,54,INDU.BA
4,Molinos Juan Semino SA,11,348,373,10,8,16,SEMI.BA
...,...,...,...,...,...,...,...,...
10393,Ten Sixty Four Ltd,20,414,554,13,18,35,X64.AX
10394,Bahrain Telecommunications Company BSC,36,149,263,9,31,68,BTEL.BH
10395,Bahrain Islamic Bank BSC,45,400,994,4,48,58,BISB.BH
10396,Ithmaar Holding BSC,38,522,994,0,30,65,ITHMR.BH


In [7]:
df.to_csv("first_draft.csv")

In [8]:
#replacing empty strings by NaN
df2 = df.replace('', np.nan)

In [9]:
#removing incomplete data
df2.dropna(how ='any',inplace=True)

In [10]:
#creating another column to sort the data according it
df2["rank/total"]=(df2["rank"].astype(int))/(df2["totalIndustries"].astype(int))

In [11]:
#sorting the data to find the upperbound
df2.sort_values(["ESG","SocialPillar","EnvironmentPillar","GovernancePillar","rank/total"],ascending=[False,False,False,False,True],inplace=True)
df2

,Name,ESG,rank,totalIndustries,EnvironmentPillar,SocialPillar,GovernancePillar,ricCode,rank/total
3412,AstraZeneca PLC,96.0,1,376,92.0,96.0,97.0,AZN.L,0.002660
3058,Roche Holding AG,95.0,2,376,94.0,97.0,92.0,ROG.S,0.005319
4799,Microsoft Corp,94.0,1,811,78.0,98.0,95.0,MSFT.O,0.001233
8376,BNP Paribas SA,94.0,1,994,94.0,96.0,93.0,BNPP.PA,0.001006
3539,GlaxoSmithKline PLC,94.0,3,376,86.0,96.0,95.0,GSK.L,0.007979
...,...,...,...,...,...,...,...,...,...
2738,Saudi Industrial Investment Group SJSC,2.0,324,325,0.0,0.0,6.0,2250.SE,0.996923
10244,Partners Value Investments LP,1.0,372,373,0.0,3.0,1.0,PVF_u.V,0.997319
6103,National Petrochemical Company SJSC,1.0,325,325,0.0,2.0,3.0,2002.SE,1.000000
10013,Netlist Inc,1.0,110,110,0.0,0.0,3.0,NLST.PK,1.000000


In [12]:
df2.reset_index(inplace=True)
df2.drop("index",axis=1,inplace=True)
df2

,Name,ESG,rank,totalIndustries,EnvironmentPillar,SocialPillar,GovernancePillar,ricCode,rank/total
0,AstraZeneca PLC,96.0,1,376,92.0,96.0,97.0,AZN.L,0.002660
1,Roche Holding AG,95.0,2,376,94.0,97.0,92.0,ROG.S,0.005319
2,Microsoft Corp,94.0,1,811,78.0,98.0,95.0,MSFT.O,0.001233
3,BNP Paribas SA,94.0,1,994,94.0,96.0,93.0,BNPP.PA,0.001006
4,GlaxoSmithKline PLC,94.0,3,376,86.0,96.0,95.0,GSK.L,0.007979
...,...,...,...,...,...,...,...,...,...
10373,Saudi Industrial Investment Group SJSC,2.0,324,325,0.0,0.0,6.0,2250.SE,0.996923
10374,Partners Value Investments LP,1.0,372,373,0.0,3.0,1.0,PVF_u.V,0.997319
10375,National Petrochemical Company SJSC,1.0,325,325,0.0,2.0,3.0,2002.SE,1.000000
10376,Netlist Inc,1.0,110,110,0.0,0.0,3.0,NLST.PK,1.000000


In [14]:
df2.to_csv("final_draft.csv")

In [16]:
dict(df2.iloc[710])

{'Name': 'Apple Inc',
 'ESG': 77.0,
 'rank': '9',
 'totalIndustries': '110',
 'EnvironmentPillar': 61.0,
 'SocialPillar': 78.0,
 'GovernancePillar': 88.0,
 'ricCode': 'AAPL.O',
 'rank/total': 0.08181818181818182}